In [ ]:
""""
Apellidos y Nombre: Zuñiga Mendez Jhonny Agustin
Carrera:Ingieneria de Sistemas
GitHub:
"""
"""
Elaborar un cuadernillo que permita demostrar la utilidad de la clasificación multiclase en el ámbito de la clasificación 
aplicada a la medicina. El código fuente del cuadernillo puede ser original o apropiado de un tercero, el código fuente 
no debe utilizar librerías o servicios de terceros, si se copia código se debe referenciar al autor. Se debe comentar 
todo el código fuente de la manera que considere apropiada el estudiante, a fin de dar la claridad suficiente en la 
explicación.
"""

In [1]:
# Clase Nodo del Árbol de Decisión
class NodoDecision:
    def __init__(self, indice_caracteristica=None, umbral=None, valor=None, rama_verdadera=None, rama_falsa=None):
        self.indice_caracteristica = indice_caracteristica  # Índice de la característica a dividir
        self.umbral = umbral  # Valor de corte para la característica
        self.valor = valor  # Etiqueta de la clase (solo para nodos hoja)
        self.rama_verdadera = rama_verdadera  # Subárbol verdadero
        self.rama_falsa = rama_falsa  # Subárbol falso

In [2]:
# Función para calcular la impureza de Gini
def impureza_gini(etiquetas):
    total_muestras = len(etiquetas)
    conteo_clases = {} #diccionario que se inicializa en vacio, se utiliza para almacenar el conteo de cada etiqueta
    for etiqueta in etiquetas:
        conteo_clases[etiqueta] = conteo_clases.get(etiqueta, 0) + 1 #devuelve el valor asociado a una clave especifica
                                                                     #si esta presente en el diccionario
    impureza = 1
    for conteo in conteo_clases.values():
        probabilidad_clase = conteo / total_muestras #calcula la probabilidad de la clase 
        impureza -= probabilidad_clase ** 2 #acumula las impuresas a medida que recorren todas las clases en el conjunto 
                                            #de datos
    return impureza

In [3]:
# Función para dividir los datos en dos ramas basadas en la característica y el valor de corte
# donde X es una lista de listas que contiene un conjunto de datos
# y Es una lista que contiene las etiquetas de clase correspondientes a cada muestra en el conjunto de datos X.
# umbral Valor de corte para la característica
#indice Es un entero que representa el índice de la característica que se utilizará para realizar la división.
def dividir_datos(X, y, indice_caracteristica, umbral):
    X_izquierda, y_izquierda, X_derecha, y_derecha = [], [], [], []
    for i in range(len(X)):#recorre cada muestra en el conjunto de datos X generando las etiquetas correspondientes
        if X[i][indice_caracteristica] <= umbral:
            X_izquierda.append(X[i])
            y_izquierda.append(y[i])
        else:
            X_derecha.append(X[i]) #etiquetas correspondientes
            y_derecha.append(y[i]) #subconjunto para construir las ramas verdaderas o falsa 
    return X_izquierda, y_izquierda, X_derecha, y_derecha

In [4]:
# Función para encontrar la mejor división que minimiza la impureza de Gini
def encontrar_mejor_division(X, y):
    mejor_gini = 1
    mejor_criterio = None  # Almacena el mejor criterio de división
    n_caracteristicas = len(X[0])#numero total de caracteristicas o columnas 

    for indice_caracteristica in range(n_caracteristicas):#recorre cada indice de caracteristica de 0 a n_caracteristicas -1
        valores_unicos = set(x[indice_caracteristica] for x in X)#Se obtienen los valores únicos para la característica actual
        for umbral in valores_unicos:#recorre cada umbral presente en los valores únicos para la característica actual
            X_izquierda, y_izquierda, X_derecha, y_derecha = dividir_datos(X, y, indice_caracteristica, umbral)
            #Se divide el conjunto de datos X y sus etiquetas y en dos subconjuntos
            gini_izquierda = impureza_gini(y_izquierda)#Se calcula la impureza de Gini para el subconjunto izquierdo
            gini_derecha = impureza_gini(y_derecha)#Se calcula la impureza de Gini para el subconjunto derecho

            # Calcula la impureza de Gini para la división
            gini = (len(y_izquierda) / len(y)) * gini_izquierda + (len(y_derecha) / len(y)) * gini_derecha

            # Actualiza el mejor criterio de división si es necesario
            if gini < mejor_gini:
                mejor_gini = gini
                mejor_criterio = (indice_caracteristica, umbral)

    return mejor_gini, mejor_criterio

In [5]:
# Función para construir el árbol de decisión recursivamente
def construir_arbol(X, y, profundidad=0, profundidad_maxima=None):
    if profundidad == profundidad_maxima or len(set(y)) == 1:
        # Crear un nodo hoja si se alcanza la profundidad máxima o todas las etiquetas son iguales
        return NodoDecision(valor=max(set(y), key=y.count))

    mejor_gini, mejor_criterio = encontrar_mejor_division(X, y)#encuentra la mejor division que minimiza la impureza de gini
    indice_caracteristica, umbral = mejor_criterio #Se extraen el índice de característica y el umbral óptimos 
    #divide el conjunto de datos en 2 subconjuntos con sus etiquetas correspondientes
    X_izquierda, y_izquierda, X_derecha, y_derecha = dividir_datos(X, y, indice_caracteristica, umbral)
    #llamada recursiva a la función construir_arbol para construir la rama verdadera del árbol utilizando el subconjunto izquierdo 
    rama_verdadera = construir_arbol(X_izquierda, y_izquierda, profundidad + 1, profundidad_maxima)
    #llamada recursiva a la función construir_arbol para construir la rama falsa del árbol utilizando el subconjunto derecho
    rama_falsa = construir_arbol(X_derecha, y_derecha, profundidad + 1, profundidad_maxima)

    return NodoDecision(indice_caracteristica=indice_caracteristica, umbral=umbral, rama_verdadera=rama_verdadera,
                        rama_falsa=rama_falsa)


In [6]:
# Función para hacer predicciones utilizando el árbol de decisión y obtener el resultado en texto
def predecir_arbol_en_texto(muestra, arbol):
    if arbol.valor is not None:#comprueba si el nodo actual del árbol (arbol) es un nodo hoja
        #si no es none significa que el nodo es un nodo hoja y contiene una etiqueta de clase (resultado) en forma numérica.
        return etiquetas_clase_a_resultados[arbol.valor]
    #Si el valor de la característica es menor o igual al umbral, se sigue la rama verdadera del árbol realizando la llamada recursiva
    if muestra[arbol.indice_caracteristica] <= arbol.umbral:
        return predecir_arbol_en_texto(muestra, arbol.rama_verdadera)
    else:
        return predecir_arbol_en_texto(muestra, arbol.rama_falsa)
etiquetas_clase_a_resultados = {
    0: "Negativo",
    1: "Positivo",
    2: "Dudoso"
}

# Conjunto de datos de entrenamiento ficticio con más características
X_entrenamiento = [[37.2, 1, 1, 1, 1],
                   [36.5, 0, 0, 1, 0],
                   [38.0, 1, 0, 0, 1],
                   [36.8, 0, 1, 0, 0],
                   [37.4, 1, 1, 1, 0],
                   [37.9, 0, 0, 0, 1],
                   [38.2, 1, 1, 1, 0],
                   [36.9, 0, 1, 1, 0]]

y_entrenamiento = [1, 0, 2, 0, 1, 0, 2, 0]

# Entrenar el modelo (Árbol de Decisión)
arbol_entrenado = construir_arbol(X_entrenamiento, y_entrenamiento, profundidad_maxima=7)

# Datos para predecir con más características
X_nuevos = [[38.7, 1, 1, 1, 1]]

# Realizar predicciones
clase_predicha = predecir_arbol_en_texto(X_nuevos[0], arbol_entrenado)

# Imprimir la clase predicha en texto
print("Clase predicha:", clase_predicha)


Clase predicha: Dudoso
